# QM/QM2 Calculation

In this example you will define a catalytic center as the bound ligand, residue 263, and a second region containing atoms within 5 Angstroms of the catalytic center. You will then set a high-level QM method for the catalytic center region, and a lower-lever QM (QM2) method to treat the other atoms. All C-alpha atoms will be constrained for optimization.

Classes used in this example:

- [GenerateModel](https://qmzyme.readthedocs.io/en/latest/API/QMzyme.GenerateModel.html)

- [QM_Method](https://qmzyme.readthedocs.io/en/latest/API/QMzyme.CalculateModel.html#qm-treatment)

- [CA_terminal TruncationScheme](https://qmzyme.readthedocs.io/en/latest/API/QMzyme.TruncationSchemes.html#QMzyme.TruncationSchemes.CA_terminal)

- [DistanceCutoff SelectionScheme](https://qmzyme.readthedocs.io/en/latest/API/QMzyme.SelectionSchemes.html#QMzyme.SelectionSchemes.DistanceCutoff)


In [5]:
import QMzyme
from QMzyme.SelectionSchemes import DistanceCutoff
from QMzyme.data import PDB

#### Initialize Model and Define Regions

In [2]:
QMzyme.data.residue_charges.update({'EQU': -1})
model = QMzyme.GenerateModel(PDB)
model.set_catalytic_center(selection='resid 263')
model.catalytic_center.set_charge(-1)
model.set_region(selection=DistanceCutoff, cutoff=5)
c_alpha_atoms = model.cutoff_5.get_atoms(attribute='name', value='CA')
model.cutoff_5.set_fixed_atoms(atoms=c_alpha_atoms)


Charge information not present. QMzyme will try to guess region charges based on residue names consistent with AMBER naming conventions (i.e., aspartate: ASP --> Charge: -1, aspartic acid: ASH --> Charge: 0.). See QMzyme.data.residue_charges for the full set.


#### Build the QM Methods and Assign to Regions

In [3]:
qm1_method = QMzyme.QM_Method(basis_set='6-31+G**', 
               functional='wB97X-D3', 
               qm_input='OPT FREQ', 
               program='orca')

qm2_method = QMzyme.QM_Method(basis_set='6-31G*', 
               functional='wB97X-D3')

qm1_method.assign_to_region(region=model.catalytic_center)
qm2_method.assign_to_region(region=model.cutoff_5)

model.truncate()

QMzymeRegion cutoff_5 has an estimated charge of -2.
QMzymeRegion catalytic_center_cutoff_5_combined has an estimated charge of -2.

Truncated model has been created and saved to attribute 'truncated' and stored in QMzyme.CalculateModel.calculation under key QMQM2. This model will be used to write the calculation input.


#### Write QM Input

In [4]:
model.write_input()

File /Users/hrk/git/QMzyme/docs/Examples/QCALC/catalytic_center_cutoff_5_combined_truncated.inp created.
